# TP : Fine-tuning de GPT-2 pour le résumé de texte en français

## Objectif
L'objectif de ce TP Est de s'inspirer de l'article [Sculpting Language: GPT-2 Fine-Tuning with LoRa](https://blog.devgenius.io/sculpting-language-gpt-2-fine-tuning-with-lora-1caf3bfbc3c6) pour effectuer le fine-tuning du modèle GPT-2 afin de réaliser des résumés de textes en français.

## Étapes

1. **Préparation du modèle**
   - Trouver un checkpoint GPT-2 pré-entraîné en français sur huggingFace.

2. **Préparation des données**
   - Identifier et obtenir un corpus de résumés de textes en français sur huggingFace.
   - Charger le corpus dans l'environnement de travail.
   - Visualiser et analyser le corpus pour s'assurer de sa qualité et de sa pertinence.

3. **Fine-tuning avec LoRA**
   - Préparer l'environnement pour utiliser LoRA (Low-Rank Adaptation).
   - Configurer les hyperparamètres pour LoRA.
   - Effectuer le fine-tuning du modèle GPT-2 avec LoRA sur le corpus de résumés.
   - Évaluer les performances du modèle fine-tuné.

4. **Fine-tuning avec QLoRA**
   - Préparer l'environnement pour utiliser QLoRA (Quantized LoRA).
   - Configurer les hyperparamètres pour QLoRA.
   - Effectuer le fine-tuning du modèle GPT-2 avec QLoRA sur le même corpus.
   - Comparer les performances et l'efficacité entre LoRA et QLoRA.

5. **Évaluation et comparaison**
   - Tester les modèles fine-tunés sur des textes non vus.
   - Analyser la qualité des résumés produits.
   - Comparer les résultats obtenus avec LoRA et QLoRA.

6. **Conclusion**
  - Faire un conclusion

#Préparation du modèle

Trouver un checkpoint GPT-2 pré-entraîné en français sur huggingFace.


In [24]:
!pip install peft
!pip install transformers
!pip install datasets

In [25]:

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModelForCausalLM.from_pretrained(
    "asi/gpt-fr-cased-base",
    device_map='auto',
)



In [26]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("asi/gpt-fr-cased-base")
tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Préparation des données

Identifier et obtenir un corpus de résumés de textes en français sur huggingFace.
Charger le corpus dans l'environnement de travail.
Visualiser et analyser le corpus pour s'assurer de sa qualité et de sa pertinence.

In [27]:
!pip install datasets

In [28]:
# LOAD AND STURCTURE DATA
from datasets import load_dataset
data = load_dataset("EdinburghNLP/orange_sum","abstract")

In [29]:
# affichage du dataset
data

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 21401
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 1500
    })
})

In [30]:
data['train'][0]

{'text': 'Thierry Mariani sur la liste du Rassemblement national (RN, ex-FN) aux européennes ? C\'est ce qu\'affirme mardi 11 septembre Chez Pol, la nouvelle newsletter politique de Libération. L\'ancien député Les Républicain et ministre de Nicolas Sarkozy serait sur le point de rejoindre les troupes de Marine Le Pen pour le élections européennes de 2019. "Ça va se faire. Ce n\'est plus qu\'une question de calendrier. On n\'est pas obligé de l\'annoncer tout de suite, à huit mois des européennes", aurait ainsi assuré un membre influent du RN. Contacté par Franceinfo, M. Mariani n\'a pas confirmé l\'information. "Les élections sont en juin, je ne sais même pas qui sera numéro 1 sur la liste", a répondu l\'ancien ministre des Transports. Il reconnaît toutefois, toujours cité par Franceinfo, que son nom sur la liste du RN "fait partie des possibilités". "Fréjus est une ville sympathique mais je n\'ai pas prévu de m\'y rendre ce week_end", a-t-il par ailleurs commenté sur Twitter alors qu

In [31]:
data['train'].to_pandas().head()

,text,summary
0,Thierry Mariani sur la liste du Rassemblement ...,L'information n'a pas été confirmée par l'inté...
1,C'est désormais officiel : Alain Juppé n'est p...,Le maire de Bordeaux ne fait plus partie des R...
2,La mesure est décriée par les avocats et les m...,"En 2020, les tribunaux d'instance fusionnent a..."
3,Dans une interview accordée au Figaro mercredi...,"Les médecins jugés ""gros prescripteurs d'arrêt..."
4,Le préjudice est estimé à 2 millions d'euros. ...,Il aura fallu mobiliser 90 gendarmes pour cett...


In [32]:
# Appliquer le tokenizer sur le texte original et le résumé
def preprocess_function(samples):
    # Tokenizer le texte d'entrée (text) avec troncation et padding à max_length
    inputs = tokenizer(
        samples['text'],
        truncation=True,
        padding='max_length',
        max_length=512
    )
    targets = tokenizer(
        samples['summary'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

    # Préparer les inputs et les labels
    inputs['labels'] = targets['input_ids']

    # Remplacer les tokens de padding des labels par -100 pour les ignorer dans la perte
    inputs['labels'] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in inputs['labels']
    ]

    return inputs

# Appliquer le tokenizer sur tout le dataset
data = data.map(preprocess_function, batched=True)


# 3-Fine-tuning avec LoRA

Préparer l'environnement pour utiliser LoRA (Low-Rank Adaptation).
Configurer les hyperparamètres pour LoRA.
Effectuer le fine-tuning du modèle GPT-2 avec LoRA sur le corpus de résumés.
Évaluer les performances du modèle fine-tuné.

In [33]:
# FREEZE WEIGHTS
for param in model.parameters():
    param.requires_grad = False

# LoRa
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.06,
    bias="none",
    target_modules=["c_attn", "c_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [34]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print_trainable_parameters(model)

trainable params: 7569408 || all params: 1024411136 || trainable%: 0.7389033303128794


In [35]:
# Importer les modules nécessaires
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback
def training(model):

    trainer = Trainer(
        model=model,
        train_dataset=data['train'],
        eval_dataset=data['test'],
        args=TrainingArguments(
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            gradient_accumulation_steps=2,
            warmup_steps=100,
            num_train_epochs=2,
            learning_rate=7e-5,
            logging_steps=100,
            save_steps=100,
            output_dir='output',
            weight_decay=0.01,
            auto_find_batch_size=True,
            evaluation_strategy="steps",
            save_strategy="steps",
            load_best_model_at_end=True,
            save_total_limit=3,
        ),
         # Use the tokenizer's pad method directly in the data collator
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Désactiver le cache pour optimiser l'utilisation de la mémoire
    model.config.use_cache = False

    # Lancer l'entraînement
    trainer.train()


In [36]:
# Training avec LOra
#training(model)

In [37]:
cd /kaggle/working/outputs/checkpoint-5350

/kaggle/working/outputs/checkpoint-5350


In [38]:
ls

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


README.md                  model_5300.zip  outputs/       trainer_state.json
adapter_config.json        model_5350.zip  rng_state.pth  training_args.bin
adapter_model.safetensors  optimizer.pt    scheduler.pt   wandb/


In [39]:
!zip -r model_5300.zip /kaggle/working/outputs/checkpoint-5300

updating: kaggle/working/outputs/checkpoint-5300/ (stored 0%)
updating: kaggle/working/outputs/checkpoint-5300/trainer_state.json (deflated 84%)
updating: kaggle/working/outputs/checkpoint-5300/README.md (deflated 66%)
updating: kaggle/working/outputs/checkpoint-5300/adapter_model.safetensors (deflated 7%)
updating: kaggle/working/outputs/checkpoint-5300/scheduler.pt (deflated 55%)
updating: kaggle/working/outputs/checkpoint-5300/rng_state.pth (deflated 25%)
updating: kaggle/working/outputs/checkpoint-5300/training_args.bin (deflated 51%)
updating: kaggle/working/outputs/checkpoint-5300/optimizer.pt (deflated 9%)
updating: kaggle/working/outputs/checkpoint-5300/adapter_config.json (deflated 52%)


In [40]:
# Enregistrer le modèle après l'entraînement
#from transformers import Trainer
#traine.save_model("model_best")

In [41]:
# Évaluer le modèle sur le jeu de données de validation 
#results = trainer.evaluate(eval_dataset=data['validation'])

# Afficher les résultats
#print("Résultats de l'évaluation :", results)


# 4- Fine tuning avec Qlora

   - Préparer l'environnement pour utiliser QLoRA (Quantized LoRA).
   - Configurer les hyperparamètres pour QLoRA.
   - Effectuer le fine-tuning du modèle GPT-2 avec QLoRA sur le même corpus.
   - Comparer les performances et l'efficacité entre LoRA et QLoRA.

In [42]:
!pip install bitsandbytes


In [43]:
import wandb
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig
import torch
from accelerate import Accelerator
import torch


# BitsAndBytes quantization config for 4-bit model loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)


# Load model and tokenizer with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    "asi/gpt-fr-cased-base",
    quantization_config=bnb_config,
    device_map='auto'
)

# FREEZE WEIGHTS
for param in model.parameters():
    param.requires_grad = False

# LoRa
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.06,
    bias="none",
    target_modules=["c_attn", "c_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

In [44]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print_trainable_parameters(model)

trainable params: 7569408 || all params: 561989120 || trainable%: 1.3468958260259558


In [45]:
# Training avec Qlora
training(model)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: Currently logged in as: kiemde-alain (datamation). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
100,2.627200,2.570312
200,2.614900,2.548828
300,2.588900,2.539062
400,2.588000,2.539062
500,2.587900,2.537109
600,2.597000,2.539062
700,2.574800,2.539062
800,2.575200,2.539062


In [46]:
# afficher le data
data['train'][1]


{'text': 'C\'est désormais officiel : Alain Juppé n\'est plus membre des Républicains. L\'ex-Premier ministre de Jacques Chirac, cofondateur de l\'UMP en 2002, ne paie plus sa cotisation auprès du parti de droite. Mercredi 9 janvier, le maire de Bordeaux a dénoncé un glissement qui s\'opère, selon lui, de la droite vers l\'extême droite. "Je me reconnais de moins en moins dans cette famille politique, à laquelle je suis pourtant très attaché (...). C\'est avec tristesse que je l\'ai quittée, mais il y a une dérive vers des thèses qui sont celles très proches de l\'extrême droite, et une ambiguïté sur l\'Europe", a-t-il déclaré face aux journalistes, réunis pour assister à ses voeux. "On assiste à cette espèce de transfusion régulière, et sur les thèmes de fond, il y a des moments où je me demande qui j\'entends à la radio ? Un membre de LR ou du RN ?", a insisté le maire de Bordeaux. Le même jour, l\'ex-député Thierry Mariani annonçait son départ de LR pour rallier une liste du Rassemb

In [47]:
data['train'][0]

{'text': 'Thierry Mariani sur la liste du Rassemblement national (RN, ex-FN) aux européennes ? C\'est ce qu\'affirme mardi 11 septembre Chez Pol, la nouvelle newsletter politique de Libération. L\'ancien député Les Républicain et ministre de Nicolas Sarkozy serait sur le point de rejoindre les troupes de Marine Le Pen pour le élections européennes de 2019. "Ça va se faire. Ce n\'est plus qu\'une question de calendrier. On n\'est pas obligé de l\'annoncer tout de suite, à huit mois des européennes", aurait ainsi assuré un membre influent du RN. Contacté par Franceinfo, M. Mariani n\'a pas confirmé l\'information. "Les élections sont en juin, je ne sais même pas qui sera numéro 1 sur la liste", a répondu l\'ancien ministre des Transports. Il reconnaît toutefois, toujours cité par Franceinfo, que son nom sur la liste du RN "fait partie des possibilités". "Fréjus est une ville sympathique mais je n\'ai pas prévu de m\'y rendre ce week_end", a-t-il par ailleurs commenté sur Twitter alors qu